<a href="https://colab.research.google.com/github/vishalrk1/tensorflow_course/blob/main/Helping_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

##Some usefule functions script 

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# importing helper function
from helper_functions import create_tensorboard_callback, plot_loss_curves, pred_and_plot, unzip_data, walk_through_dir

## downloading and unzipping zip files of data set

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

## unzip
zip_ref = zipfile.ZipFile('pizza_steak.zip')
zip_ref.extractall()
zip_ref.close()

--2021-10-04 16:25:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 173.194.69.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109579078 (105M) [application/zip]
Saving to: ‘pizza_steak.zip.1’

pizza_steak.zip.1   100%[===================>] 104.50M   195MB/s    in 0.5s    

2021-10-04 16:25:18 (195 MB/s) - ‘pizza_steak.zip.1’ saved [109579078/109579078]



##Files in directory

In [ ]:
import os

for dirpath, dirnames, filename in os.walk('pizza_steak'):
  print(f"there are{len(dirnames)} directories and {len(filename)} images in '{dirpath}' ")

there are2 directories and 1 images in 'pizza_steak' 
there are2 directories and 1 images in 'pizza_steak/train' 
there are0 directories and 750 images in 'pizza_steak/train/pizza' 
there are0 directories and 750 images in 'pizza_steak/train/steak' 
there are2 directories and 1 images in 'pizza_steak/test' 
there are0 directories and 250 images in 'pizza_steak/test/pizza' 
there are0 directories and 250 images in 'pizza_steak/test/steak' 


## Baseline model for CNN Image classification

In [ ]:
model_1 = Sequential([
    Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
    MaxPool2D(),
    Conv2D(10, 3, activation='relu'),
    MaxPool2D(),
    Conv2D(10, 3, activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(10, activation='softmax'),
])

model_1.compile(
    loss='categorical_crossentropy',
    optimizer = Adam(),
    metrics = ['accuracy'],
)

history_1 = model_1.fit(
    train_data_argumented,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data),
)

##Function to plot loss and accuracy curves from history of model

In [ ]:

# ploting validation and trainning curve separately
def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss =  history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epoch = range(len(history.history['loss']))

  # plot loss
  plt.plot(epoch, loss, label='training_loss')
  plt.plot(epoch, val_loss, label='val_loss')
  plt.title('loss')
  plt.xlabel('epochs')
  plt.legend()

  # plot accuracy
  plt.figure()
  plt.plot(epoch, accuracy, label='training_accuracy')
  plt.plot(epoch, val_accuracy, label='val_accuracy')
  plt.title('accuracy')
  plt.xlabel('epochs')
  plt.legend()

## Data agumentation

* Improve Data accuracy
* Reduce overfitting ( get the train and validation loss curve cloer)

In [ ]:
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=0.2,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True)

train_data_argumented = train_datagen_augmented.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               class_mode = 'categorical',
                                               batch_size = 32)

###Data Agumentation using image_dataset_from_directory

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE= 32

print('Training Data')
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory = train_dir,
    image_size = IMG_SIZE,
    label_mode = 'categorical',
    batch_size = BATCH_SIZE,
)

print('Test Data')
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = test_dir,
    image_size = IMG_SIZE,
    label_mode = 'categorical',
    batch_size = BATCH_SIZE,
)

###Data Augmentation layer using sequential model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# data augmentation stage with horizontal flipping, rotation, zooms, etc....
data_augmentation = keras.Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    # preprocessing.Rescaling(1./255)
], name='data_augmentation')

##Function to plot and make predictions on our images 

In [ ]:
def pred_and_plot(model, filename, class_names):

  # loading and preparing image
  img = load_prep_image(filename)

  # predicting
  pred = model.predict(tf.expand_dims(img, axis=0))

  # get predicted class
  pred_class = class_names[int(tf.round(pred))]

  # ploting image
  plt.imshow(img)
  plt.title(f"prediction: {pred_class}")
  plt.axis(False);

* **Loading and preparing images for predictions**

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.io.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

#Feature extraction

##Setting up callbacks

* Tracking experiments with **Tensor Board callBack**
* Model checkpoint with **Model check Callback**
* Stopping a model from training ( before it trains to long and overfits) with the **Early Stopping Callback** 

###Tensorboard Callback

In [ ]:
# creating tensorflow callback
import datetime

def creat_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime("%d%m%Y-&H&M")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"aving Tensorboard log file to: {log_dir}")
  return tensorboard_callback

# Tensorflow Hub

##Creating a function to creat model from tensorflow hub link

In [ ]:
def creat_model(model_url, num_classes=10):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """

  # Downloading the pretrained model and saveing it as a keras layer
  feature_exrtraction_layer = hub.KerasLayer(model_url, 
                                             trainable=False, 
                                             impute_shape=IMAGE_SHAPE + (3,),
                                             name='feature_extraction_layer')
  # Creating model
  model = Sequential([
     feature_extraction_layer,
     Dense(num_classes, activation='softmax', name='output_layer') 
  ])

  return model